In [1]:
#from diy_lib.decision_tree import Decision_tree
#from diy_lib.node import Node
import pandas
from typing import List
from itertools import combinations
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

In [67]:
import itertools
import numpy as np
class Node:
    """hi"""
    next_id = 0         
    def __init__(self,parent, level, data = None ,feature=None,threshold=None,gini_val=None,most_common_class=None):
        self.most_common_class=most_common_class
        self.parent = parent
        self.level = level
        self.gini_val = gini_val #none as defualt
        self.threshold = threshold #none as defualt
        self.left_child = None #none as defualt
        self.right_child = None #none as defualt
        self.feature=feature
        self.id = Node.next_id
        self.data = data
        Node.next_id += 1

    def create_children(self,lchild,rchild):
        """
        Attaches the children to the node
        """
        self.left_child = lchild
        self.right_child = rchild
    
    def get_children(self):
        """
        Returns the children of the node
        """
        return self.left_child, self.right_child
    
    def get_position(self):
        """
        Returns the position in the tree,
        and whether or not it is a root or leaf
        """
        if self.parent==None:
            return("root")
        elif self.left_child == None:
            return('leaf')
        else:
            return f"node at depth {self.level}"

    def get_info(self):
        if self.parent == None:
            parent_id = None
        else:
            parent_id = self.parent.id
        if self.get_children()[0] == None:
            children_id = [None,None]
        else:
            children_id=[self.get_children()[0].id,self.get_children()[1].id]
        return f"ID: {self.id} \nParent: {parent_id} \nChildren: {children_id}\nMost_common_class: {self.most_common_class} \nFeature: {self.feature} \nThreshold: {self.threshold} \nGini: {self.gini_val} \nDepth: {self.level} \nPosition: {self.get_position()}\nAmount of Datapoints: {len(self.data)}\nClasses of Datapoints: {Counter(self.data['type'])}"
        


def main():
    pass
if __name__=="__main__":
    main()

In [57]:
test=pandas.read_csv("../data/df_test.csv")
train=pandas.read_csv("../data/df_train.csv")
#train

In [4]:
print(len(train))

149


In [99]:
class Decision_tree:
    def __init__(self,depth,min_samples_split=0):
        self.depth = depth
        self.min_samples_split=min_samples_split
        self.tree = {}
    def _splitable(self,current_n,data):#end conditions where we dont split and instead we make leaves 
        if current_n.level == self.depth:
            return False
        elif len(data)<self.min_samples_split:
            return False
        else:
            return True
    def _create_tree(self, current_n,tree, data):
        #should we split?
        """This function is creating nodes though recursion"""
        if self._splitable(current_n,data)==False:
            print(current_n,"hi")
            pass
        #we are splitting 
        else: #creates 2 children when not a leaf
            split = self.find_split(data) #(gini_min, threshold, best_feature)
            current_n.gini_val = split[0]
            current_n.threshold = split[1]
            current_n.feature = split[2]

            mask = data[split[2]]>split[1]
            rightsplit = data[mask]
            leftsplit = data[-mask] #check right split on feature contains higher values 
            l_child = Node(parent = current_n, data = leftsplit, gini_val = self._gini_oneside(leftsplit["type"]) ,level = current_n.level+1,most_common_class=max(Counter(leftsplit["type"]), key = Counter(leftsplit["type"]).get)) # < threshold
            self.tree[l_child.id] = l_child
            r_child = Node(parent = current_n, data = rightsplit, gini_val = self._gini_oneside(rightsplit["type"]) ,level = current_n.level+1,most_common_class=max(Counter(rightsplit["type"]), key = Counter(rightsplit["type"]).get)) # > threshold
            self.tree[r_child.id] = r_child
            current_n.create_children(l_child,r_child)
            self._create_tree(l_child,self.tree,data = leftsplit)
            self._create_tree(r_child,self.tree,data = rightsplit)
        return self.tree

    def _gini_oneside(self,data):
        """Takes a list of labels for one side of a 
        split and returns gini index for that side"""
        size = len(data)
        types = set(data)
        represented = Counter(data)
        gini = 1
        for key, value in represented.items():
            gini -= (value/size)**2
        return gini

    def _gini_overall(self,datal,datar):
        lsize = len(datal)
        rsize = len(datar)
        size = lsize+rsize
        return self._gini_oneside(datal)*(lsize/size)+self._gini_oneside(datar)*(rsize/size)

    def fit(self,data):
        self.data = data
        Node.next_id = 0
        root = Node(parent=None, level = 0,data=self.data,most_common_class=max(Counter(data["type"]), key = Counter(data["type"]).get))
        self.tree[root.id] = root
        self.tree = self._create_tree(root,self.tree,self.data)
    
    def view(self):
        """
        Returns the tree
        """
        return self.tree
    
    def _split(self):
        #Make the split
        #calculate the threshold, for each child
        #calculate the gini value
        return #return two tuples, contaning: (feature, threshold, gini_val)

    def find_split(self,data):
        results=[]
        gini_min = 1000
        threshold = None
        best_feature= None
        for feature in data.loc[:, data.columns != "type"]:
            sl = list(set(data.sort_values(feature)[feature]))         #sort data by feature
            list_PS = [(sl[i+1]+sl[i])/2 for i in range(len(sl)-1)]         #find betweeen values for each pair of values in sorted 
            for PS in list_PS:#itterate through between values instead 
                mask = train[feature]>PS
                split = list(data[mask]["type"])
                antisplit = list(data[-mask]["type"]);
                gini_val = self._gini_overall(split,antisplit)
                if gini_val < gini_min:
                    gini_min = gini_val
                    threshold = PS 
                    best_feature = feature

                results.append([PS, self._gini_overall(split,antisplit),feature])
        return (gini_min, threshold, best_feature)
    def predict(self, data):
        predictions = []
        for point in data.values:
            leaf_node = self._traverse(self.tree[0],point)
            predictions.append(leaf_node.most_common_class)
        return predictions
        pass
    def _traverse(self,current_node,point):
        #move along decision tree until leaf is reached
        if current_node.left_child == None :
            return current_node
        else:
            threshold=current_node.threshold
            feature=current_node.feature 
            print("Hello",point)
            chosen_child=None
            return self._traverse(chosen_child,point)

        #if leaf then return leaf
        #else find next node
        return None
        pass

In [ ]:
#to do
#add most common class to root
#add most common class to all created children
#add to get info

#make traverse function
#finish and check predict function
#test on train and test as comparrison

In [86]:
max(Counter(train["type"]), key = Counter(train["type"]).get)

2

In [97]:
train.values

array([[ 1.51839, 12.85   ,  3.67   , ...,  0.     ,  0.35   ,  2.     ],
       [ 1.52081, 13.78   ,  2.28   , ...,  0.     ,  0.17   ,  2.     ],
       [ 1.51708, 13.72   ,  3.68   , ...,  0.     ,  0.     ,  2.     ],
       ...,
       [ 1.52365, 15.79   ,  1.83   , ...,  1.68   ,  0.     ,  7.     ],
       [ 1.51571, 12.72   ,  3.46   , ...,  0.     ,  0.24   ,  1.     ],
       [ 1.51786, 12.73   ,  3.43   , ...,  0.     ,  0.3    ,  1.     ]])

In [90]:
T.predict(test)

Hello RI


AttributeError: 'NoneType' object has no attribute 'left_child'

In [89]:
T=Decision_tree(3,5)
T.fit(train)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:84: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:85: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:84: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:85: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:84: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:85: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/shared-libs/python3.7/py-core/lib

In [78]:
print(T.view()[1].get_info())

ID: 1 
Parent: 0 
Children: [3, 4]
Most_common_class: 2 
Feature: Al 
Threshold: 1.415 
Gini: 0.605078125 
Depth: 1 
Position: node at depth 1
Amount of Datapoints: 128
Classes of Datapoints: Counter({2: 52, 1: 48, 3: 12, 5: 8, 6: 6, 7: 2})


In [54]:
print(T.view()[9].most_common_class)

7


In [ ]:
tree=Decision_tree(train,depth=3)
tree.create_tree()
tree.view()

TypeError: __init__() got multiple values for argument 'parent'

In [ ]:
A = Node("bob",parent = None, level = 0)
print(A.get_position())

root


In [ ]:
_deepnote_run_altair(train, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"Na","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"Mg","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
_deepnote_run_altair(train, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"RI","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"Na","type":"quantitative","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
#To delete 8.12.21
def create_tree(max_depth):
    tree={}
    Node.next_id=0
    root=Node(parent = None, level = 0)
    tree[root.id]=root
    tree=rec(root, max_depth,tree)
    return tree

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58866f15-1cff-4fef-9525-5c3070562370' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>